# Exploring the Ringelmann effect with git2net
**Lucas Schelkes**



In this project, we analyze the effect of group size on the productivity in GitHub projects. 

Using the tool `git2net`, we will analyse the following GitHub repositories:

1. `TLP` (power saving tool for ThinkPads) as a small example (11 committers)

2. `Android` (Android architecture blueprints v2 for building apps) as an medium-sized example (40 committers)

3. `nginx-proxy` (sets up container for running nginx) as an medium-large-sized example (60 committers)

4. `hello-world` ("hello world" code in all languages) as a large-sized example (79 committers)

Finally we will derive to an interpretation of the analysis in respect to the Ringelmann effect.

## `TLP` repo

We first mine the co-editing data from variously sized git repositories.
First, we specify the `sqlite` database location and the to-be-mined cloned `GitHub` repository.

In [1]:
import os
import git2net

#path to copy of virgin db:
sqlite_db_file = '/home/luc/pip/git2net/group_work_1/tlp (Kopie).db'
#sqlite_db_file = '/home/luc/pip/git2net/group_work_1/bitcoin.db'
#sqlite_db_file = '/home/luc/pip/git2net/group_work_1/architecture-samples (Kopie).db'



#set variable to whatever folder the to-be-mined git repository is located:
#repo_dir = '/home/luc/pip/git2net/ds_gw_1/TLP'
#repo_dir = '/home/luc/pip/git2net/group_work_1/architecture-samples'

## Mining

Only do the mining, if a copy of the "virgin" sqlite database has not being saved in `sqlite_db_file`.
This copy needs to be freshly recopied from the virgin file in every Notebook run-through to avoid database errors.

In [2]:
#uncomment this block, if mining is wished

#check, if database file is there; remove if yes
#if os.path.exists(sqlite_db_file):
#    os.remove(sqlite_db_file)
#    print('sqlite_db_file removed!')

#mining process: 
#repo_dir: to-be-mined git repository
#sqlite_db_file: to-be-stored values in database

#git2net.mine_git_repo(repo_dir, sqlite_db_file)

#git2net.mining_state_summary(repo_dir, sqlite_db_file)

## Visualization

In order to get a first idea of who about did roughly what with whom else, lets look at a graph showing edges from `committer_A` to `committer_B` whenever `committer_A` changed a line of the original author `committer_B`.

In [3]:
#package for network visualization:
import pathpy as pp

#fetching information:
t, node_info, edge_info = git2net.get_coediting_network(sqlite_db_file)

#showing graph:
pp.Network.from_temporal_network(t)

This already gives us the idea that `Thomas Koch` is the centre figure of the community.

Lets look at the work activity of the committers in the last 6 months, this gives us an idea who has recently been active. The following graph shows us which committer has edited which files from `01.05.2019` onwards.

In [4]:
from datetime import datetime

time_from = datetime(2019, 5, 1)
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file, time_from=time_from)

n = pp.Network.from_temporal_network(t)
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_colour = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_colour)

Again, `Thomas Koch` seems not only to be the centre figure of the community, but also one of the only few with recent activity, doing most of the work himself.

## Sqlite queries

Lets begin to analyse the sqlite database, by connecting to the file.

In [5]:
import pandas as pd
import sqlite3

#connect to database file:
con = sqlite3.connect(sqlite_db_file)
c = con.cursor()

Since we only want to look at the edit type `replacement` in the `edits`-table, lets create a new table `df2` with the columns that we need for linear regression, namely:

1. the distinct `hash` commit value
2. the `committer_name`
3. the `committer_date`
4. the parent commit hash `original_commit_deletion`
5. and the producitivity measure `levenshtein_dist`.

In [6]:
#create a new table with the desired column names:
query0="""
    CREATE TABLE IF NOT EXISTS df2(
    hash TEXT PRIMARY KEY,
    committer_name TEXT,
    committer_date TEXT,
    original_commit_deletion TEXT,
    levenshtein_dist INTEGER);
"""

#execute the queries on the database:
c.execute(query0)

Following task 02, lets select only rows of table `edits`, where `edit_type='replacement'` and merge this with table `commits` along the primary key `hash`. This gives us a joined table of `edits` and `commits`, but with hashes occuring multiple times, each with different `levenshtein_dist`.

Therefore we group this query by unique hashes, taking the sum of the `levenshtein dist` of same hashes, because we want to know the total `levenshtein_dist` for each commit and not for every single edit.

Finally, we insert this into the new table `df2` to avoid key words (e.g. `SUM(levenshtein_dist)`) as column names.

In [7]:
query1="""
    INSERT INTO df2(hash, committer_name, committer_date, original_commit_deletion, levenshtein_dist)
    SELECT hash, committer_name, committer_date, original_commit_deletion, "SUM(levenshtein_dist)"
    FROM
    (
    SELECT 
    hash,
    committer_name,
    committer_date,
    original_commit_deletion,
    SUM(levenshtein_dist)
    FROM(SELECT * FROM edits INNER JOIN commits ON commits.hash = edits.commit_hash WHERE edit_type = 'replacement')
    GROUP BY hash
    );
"""

c.execute(query1)

We can count the distinct `committer_names`, this gives us an idea of the size.

In [8]:
#count the distinct committer names:
query1_1="""
    SELECT count(DISTINCT committer_name) FROM df2;
"""

c.execute(query1_1)

#store amount of distinct committer names as int...
number_committers = c.fetchone()[0]

print('The number of committers we are dealing with is', number_committers)

#save the queries:
con.commit()

The number of committers we are dealing with is 11


Lets take a look of the committers names with the following query.

In [9]:
#select the distinct committer names
query1_2="""
     SELECT DISTINCT committer_name FROM df2;
"""

c.execute(query1_2)

#store names of committers in list:
committer_list = c.fetchall()

#save the queries:
con.commit()

#close the database to make it accessible for others:
con.close()

In [10]:
#turn list of string-tuples into list of single-strings:
committers = [i[0] for i in committer_list]

print('The committers names are:\n')

committers


The committers names are:



['Thomas Koch',
 'GitHub',
 'SammysHP',
 'André Erdmann',
 'Aaditya Bagga',
 'Timofey Titovets',
 'Connor Prussin',
 'TK',
 'Kai-Heng Feng',
 'Maxime Gauduin',
 'Qiang Yu']

We want to find a productivity measure that suits each commit. The idea is to find the next commit, that an author did, and calculate the `levensthein_dist` for that specific time intervall between the two commits. Therefore, we first need to sort the `df2`-table by ascending `commit_dates`.

We call this new table `df3`. We do this by executing our sorting query on `df2` and transfering everything to `df3`. By doing this procedure we avoid table errors and unclean workspace.

In [11]:
#create new empty table:
query01="""
    CREATE TABLE IF NOT EXISTS df3(
    hash TEXT PRIMARY KEY,
    committer_name TEXT,
    committer_date TEXT,
    original_commit_deletion TEXT,
    levenshtein_dist INTEGER);
"""

#sort committer_date ascending:
query2="""
    INSERT INTO df3(hash, committer_name, committer_date, original_commit_deletion, levenshtein_dist)
    SELECT *
    FROM
    (
    SELECT * 
    FROM df2 
    ORDER BY committer_date ASC
    );
"""

#delete table df2:
query3="""
    DROP TABLE df2;
"""

con = sqlite3.connect(sqlite_db_file)
c = con.cursor()

#execute the queries:
c.execute(query01)
c.execute(query2)
c.execute(query3)
#result: df3

#close connection:
con.commit()
con.close()

## Numpy array

Now we have a clean table with ascending `committer_dates`. We can transform this into a numpy array for faster and more precise calculations. 

In [12]:
#reopen connection:
con = sqlite3.connect(sqlite_db_file)

#output df2 as a pandas dataframe:
df = pd.read_sql_query("SELECT * FROM df3;", con=con)

#turn Pandas dataframe into numpy array:
df = df.values

print('The first row of the np array: \n', df[0])

The first row of the np array: 
 ['4b6204169763d76567c846528be692afd30c38e4' 'Thomas Koch'
 '2010-01-22 18:16:44' '624dc813bd780af99e0d73e31fd2de9f3cc6c9ad' 4]


## Productivity measure

Since we will need to extend the array for the extra productivity value that we will calculate, we introduce a small function that appends further columns to our array.

In [13]:
import numpy as np

#function for adding a full-length '0'-column to the dataframe:
def add_column(df):
    new_column = np.array(np.zeros(df.shape[0]))
    return np.column_stack((df, new_column))

In [14]:
#add column:
df = add_column(df)
print('Shape of array:', df.shape)

Shape of array: (942, 6)


In order to find out the total time between two commits of each author, we must turn the `committer_date` (string) into a readible numeric object. We can do this by stripping the time variables and returning a `datetime`-object.

In [15]:
#make date stamps readible, by turning them into a datetime-object:

from datetime import datetime

def date_to_int(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

print(date_to_int(df[0][2]))
print(type(date_to_int(df[0][2])))

2010-01-22 18:16:44
<class 'datetime.datetime'>


Since the table is sorted by ascending `committer_dates`, we can simply run a loop and check for the next commit that a certain author delivered. We then calculate the amount of time (in seconds) between the two commits; we save the value in the newly created column.



In [16]:

for i in range(df.shape[0]-1):
    #grab the author and his commit_time:
    author = df[i][1]
    commit_time = df[i][2]
    for j in range(i+1,df.shape[0]):
        #search for the his next commit:
        author_next = df[j][1]
        commit_time_next = df[j][2]
        #if there is a "next commit", calculate the commit_time differences (in total seconds) and save the value in the last column:
        if author == author_next:
                #calulate the total time (in seconds) between the 2 commits:
                df[j][5] = (date_to_int(commit_time_next) - date_to_int(commit_time)).total_seconds()
                break
            

In [17]:
#add empty 6th column:
df = add_column(df)
print('Shape of array:', df.shape)

Shape of array: (942, 7)


If someone has a `levensthein_dist` of 1000, but took 3 months to commit it, he was not very productive.
Whereas someone with that same `levensthein_dist`, who commited in 3 days, is far more productive. 

Therefore, we define the productivity measure for each authors commit to be the `levensthein_dist` per second:

$$\text{levensthein_dist/second} \quad = \quad \frac{\text{levensthein_dist}}{\text{time_between_two_commits}}$$ 

In [18]:
for i in range(df.shape[0]):
    #if time_between_commits = 0, then set productivity to 0
    if df[i][5] == 0:
        df[i][6] = 0
    #else, calculate productivity measure and store in 6th column
    else:     
        df[i][6] = df[i][4] / df[i][5]

In [19]:
#add empty 7th column:
df = add_column(df)
print('Shape of array:', df.shape)

Shape of array: (942, 8)


## Collaboration variable

Following task 03, we want to construct a variable defining if the author collaborated or not for his commit, i.e. whether or not he is editing someone elses commit.

We set a boolean variable
- '1' if the authors parents hash is another authors commit hash (collaborated: True)
- '0' if this is not the case (collaborated: False)

and store this in the newly created column.

In [20]:
for i in range(df.shape[0]):
    parent_hash = df[i][3]
    #set orig_author to default 'nobody'
    #(important for genesis commit hashes)
    orig_author = 'nobody'
    for j in range(i):
        #check whether any other prior hash is from the same author
        if df[j][0] == parent_hash:
            #update orig_author with real author name:
            orig_author = df[j][1]
            #if original hash found, then break:
            break
    #set a place holder in the 7th column:
    df[i][7] = 'no parent_hash'
    #if author collaborated, save '1', if not, save '0':
    #only if orig_author was another author, did the author collaborate:
    if orig_author != df[i][1] and orig_author != 'nobody':
        df[i][7] = 1
    else:
        df[i][7] = 0

Now check the `df`-variable. We know have all the data in the columns needed for linear regression!

Specifically, we only need

1. the `committer_name`, capturing the identity of the developer. Each developer is obviously differently skilled, so we want to capture this effect in a specified variable when doing linear regression

2. the `levenshtein_dist/second`, measuring the productivity of every authors commit

3. the `collab_bool`, telling us whether or not the author "collaborated"

We delete every other column of our array.

In [21]:
#delete commit_hash:
df = np.delete(df,0,1)
#delete commit_time:
df = np.delete(df,1,1)
#delete parent_hash:
df = np.delete(df,1,1)
#delete levensthein_dist:
df = np.delete(df,1,1)
#delete time_between_commits:
df = np.delete(df,1,1)

print('Shape of array:', df.shape)

Shape of array: (942, 3)


Its most convenient to perform (multiple) linear regression using `statsmodels`. This works best if we use a `pandas` dataframe, since we can call columns simply by their names. So in the following, we convert our `numpy` array into a 3-column `pandas` dataframe.

In [22]:
#insert first column of df as 'committer_names':
pdf = pd.DataFrame({'committer_names': df[:,0]})
#insert second column of df as 'productivity_per_second':
pdf['productivity_per_second'] = df[:,1]
#we must account for the lack of pandas' ability to handle scientific numeric notation (e.g. e^-5) by specifically turning them into numeric values:
pdf['productivity_per_second'] = pd.to_numeric(pdf['productivity_per_second'])
#insert third column of df as 'collab_bool':
pdf['collab_bool'] = df[:,2]

print(pdf.shape)
print(pdf.head(3))

(942, 3)
  committer_names  productivity_per_second collab_bool
0     Thomas Koch                 0.000000           0
1     Thomas Koch                 0.000099           0
2     Thomas Koch                 0.007752           0


## Multiple linear regression

Finally we can run linear regression!

We must keep in mind, that the `committer_names` are `strings` and for each `committer_name`, we want to have a variable that captures the developers identity. Since they are 'strings', it would make them `categorical` variables. We can captures all of the different `committer_names` by introducing `dummy` variables. 

Thankfully, `statsmodels` sports an useful feature for automatically dealing with turning these `categorical` into `dummy` variables. 
We can neglect any common influence of one certain `committer_name` with another.

Our multiple linear regression therefore consists of $$\beta_{\text{col}} \cdot \text{collab_bool} \quad + \quad \textbf{beta}^{\text{T}} \cdot \textbf{committer_names} \quad = \quad  \text{productivity_per_second},$$
where $\textbf{beta} = (\beta_0, \ldots, \beta_{\text{number_committers}+1})$, ($\beta_0$ accounts for the intercept) 

$\textbf{committer_names} = (\mathbb{1}, \text{committer_names}[0], \ldots, \text{committer_names}[\text{number_committers}])$

By investigating the value of the estimated $\beta_{\text{col}}$, we can explain the linear effect of collaborating on the productivity. This may allow us to discuss the Ringelmann effect when comparing different group sizes.


In [23]:
import statsmodels.formula.api as sm

# format: y ~ x + committer_names:
model = sm.ols(formula='productivity_per_second ~ collab_bool + committer_names', data = pdf)
#train model:
fit = model.fit()

#print results:
print(fit.summary())

OLS Regression Results                              
Dep. Variable:     productivity_per_second   R-squared:                       0.008
Model:                                 OLS   Adj. R-squared:                 -0.004
Method:                      Least Squares   F-statistic:                    0.6453
Date:                     Sat, 16 Nov 2019   Prob (F-statistic):              0.790
Time:                             22:15:48   Log-Likelihood:                -2825.6
No. Observations:                      942   AIC:                             5675.
Df Residuals:                          930   BIC:                             5733.
Df Model:                               11                                         
Covariance Type:                 nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

## Summary of `TLP` repo

We can see all the valuable information in the summary. Aside of giving us the calculated intercept and the various slope coefficients, we may also interpret other useful parameters.

The negative `collab_bool` value could suggest a negative impact on productivity if committers are collaborating, but: we defined this value to simply be a `True` or `False` value whether or not the committer is working on a commit of another author. This is simply not strong enough for explaining real collaboration! It would need at least 2 back-and-fourth commits of two authors to be able to call it a collaboration. 
Furthermore, the estimated `collab_bool` value of $-1.0798$ does not seem robust, since we know from the visualizations that basically only Thomas Koch did all the work, no real collaboration can be found in the data. Any other of the few authors editing files of Thomas Koch is distorting this value, since they are not way as productive as lone wolf Thomas Koch is. This negative `collab_bool` is not robust enough to support a decline of productivity for collaborating team members. Significantly large confidence intervalls further nourish this hypothesis. 

In addition, we observe fairly high p-values in the summary for almost all coefficients, so we cannot reject the hypothesis, that there is no linear relationship in the data, thus we may assume that there is no linear relationship at all!

In order to even prove a linear relationship in the data, the assumption of normally distributed residuals must hold. However, we need to check the normality of the residuals to test for a linear relationship. We can immediately see that $\text{Prob(Omnibus)}=0$ and $\text{Prob(JB)}=0$, this allows to confidently reject the hypothesis, that the residuals are normally distributed. Summarising the analysis, we can assume that there is no linear relationship in the data, thus we cannot properly discuss the negative `collab_bool` in respect to the Ringelmann effect.

## Predictions

We can now construct fictive predictions, by creating a dataframe with the corresponding exog values.

In [24]:
#In Sample Predictions:
print('The predictions of the first 5 samples in the dataset [Thomas Koch, False]:\n', fit.predict(pdf).head())
print('\n')
print('The predictions of [TK, True] (pdf[206]):',fit.predict(pdf)[206])
print('\n')
print('The predictions of [TK, False] (pdf[207]):',fit.predict(pdf)[207])
print('\n')
print('The predictions of [Connor Prussin, True] (pdf[324]):',fit.predict(pdf)[324])

The predictions of the first 5 samples in the dataset [Thomas Koch, False]:
 0    0.304082
1    0.304082
2    0.304082
3    0.304082
4    0.304082
dtype: float64


The predictions of [TK, True] (pdf[206]): -0.35528465888192673


The predictions of [TK, False] (pdf[207]): 0.7244880110708445


The predictions of [Connor Prussin, True] (pdf[324]): 1.031628700921914e-14


In [25]:
#Construct prediction for: [Thomas Koch, True]
X_pred = pd.DataFrame({'committer_names': 'Thomas Koch'}, index=[0])
X_pred['collab_bool'] = 1

pred = fit.predict(X_pred)
pred[0]

-0.7756908380888659